# Interactive Weir Flow Profile with Flow Classification

This notebook calculates the free water surface profile upstream of a weir and provides critical depth analysis with interactive widgets.
You can adjust flow parameters, select weir type, and visualize changes in real-time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display, clear_output

g = 9.81  # gravitational acceleration (m/s^2)

In [2]:
def compute_critical_depth(Q, b):
    return (Q**2 / (g * b**2))**(1/3)

def compute_normal_depth(Q, b, n, S0):
    func = lambda y: (1/n) * b * y**(5/3) * np.sqrt(S0) - Q
    y_guess = (Q**2 / (g * b**2))**(1/3) * 1.5
    return fsolve(func, y_guess)[0]

def Sf(y, Q, b, n):
    A = b * y
    R = y
    return (n**2 * Q**2) / (A**2 * R**(4/3))

def Fr(Q, b, y):
    return Q / (b * y * np.sqrt(g * y))

def flow_classification(y, y_crit):
    if y > y_crit:
        return 'Subcritical'
    elif y < y_crit:
        return 'Supercritical'
    else:
        return 'Critical'

In [3]:
def plot_profile(Q, b, n, S0, L, dx, weir_type, weir_height, C_b, C_d):
    y_crit = compute_critical_depth(Q, b)
    y_normal = compute_normal_depth(Q, b, n, S0)
    y_start = y_crit * 1.05

    x_vals = [0]
    y_vals = [y_start]
    x, y = 0, y_start

    while x < L:
        fr = Fr(Q, b, y)
        denom = 1 - fr**2
        if abs(denom) < 1e-6:
            break
        dy_dx = (S0 - Sf(y, Q, b, n)) / denom
        y += dy_dx * dx
        x += dx
        x_vals.append(x)
        y_vals.append(y)

    if weir_type == 'broad-crested':
        Q_weir = C_b * b * y_crit**1.5
    else:
        Q_weir = (2/3) * C_d * b * np.sqrt(2 * g) * y_crit**1.5

    plt.figure(figsize=(10, 5))
    plt.plot(x_vals, y_vals, label='Water Surface')
    plt.axhline(y_crit, color='r', linestyle='--', label='Critical Depth')
    plt.axhline(y_normal, color='g', linestyle=':', label='Normal Depth')
    plt.title(f'Flow Profile ({weir_type})')
    plt.xlabel('Distance (m)')
    plt.ylabel('Depth (m)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print(f"Critical Depth: {y_crit:.3f} m")
    print(f"Normal Depth: {y_normal:.3f} m")
    print(f"Flow Type at Start: {flow_classification(y_start, y_crit)}")
    print(f"Estimated Weir Discharge: {Q_weir:.3f} m³/s")

In [4]:
interact_ui = widgets.interactive(
    plot_profile,
    Q=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Discharge Q (m³/s)'),
    b=widgets.FloatSlider(value=3.0, min=1, max=10, step=0.5, description='Channel Width (m)'),
    n=widgets.FloatSlider(value=0.03, min=0.01, max=0.1, step=0.005, description="Manning's n"),
    S0=widgets.FloatLogSlider(value=0.001, base=10, min=-4, max=-1, step=0.1, description='Slope S₀'),
    L=widgets.FloatSlider(value=100.0, min=10, max=300, step=10, description='Length (m)'),
    dx=widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='dx step (m)'),
    weir_type=widgets.Dropdown(options=['broad-crested', 'sharp-crested'], value='broad-crested', description='Weir Type'),
    weir_height=widgets.FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='Weir Height (m)'),
    C_b=widgets.FloatSlider(value=1.7, min=1.0, max=2.5, step=0.1, description='C_b (broad)'),
    C_d=widgets.FloatSlider(value=0.6, min=0.4, max=0.9, step=0.05, description='C_d (sharp)')
)

display(interact_ui)

interactive(children=(FloatSlider(value=10.0, description='Discharge Q (m³/s)', max=30.0, min=1.0, step=1.0), …